In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.environ.get('PATH_CUSTOM_MODULES'))

import augment_image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Prepare all basic variable

In [ ]:
path_source = os.environ.get('PATH_DATASET_DESTINATION')
path_img = os.environ.get('PATH_DATA_AUG_SAMPLE')
scenario_names = ['scenario_1', 'scenario_2', 'scenario_3', 'scenario_4', 'scenario_5']
dataset_names = ['rimone', 'g1020', 'refuge', 'papila']
fold = 'fold_2'
labels_name = ['normal', 'glaukoma']
aug_type = ['h_flip', 'v_flip', 'bright']
image_size = {'rimone': (300,300),
            'g1020': (240,300),
            'refuge': (300,300),
            'papila': (200,300)}

In [ ]:
# getting the path of the dataset
path_dataset_src = {}
for scenario in scenario_names:
    for dataset in dataset_names:
            path_dataset_src[scenario + '_'
                            + dataset + '_'
                            + fold] = os.path.join(path_source,
                                                    scenario,
                                                    dataset,
                                                    fold,
                                                    'train')
del scenario, dataset

In [ ]:
# create the image data generator for original image
datagen = ImageDataGenerator(
    rescale=1./255,
)
# create the image data generator for clahe image
datagen_clahe = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=augment_image.clahe
)

### Visualize the Image
#### Scenario 1
**Condition:**
- no augmentation
- rgb color
- no clahe

##### Prepare the needed variable

In [ ]:
s1_datagen = {}
s1_col_mode = 'rgb'
s1_aug = 'no'

##### Import the image into data datagenerator

In [ ]:
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s1_datagen[dataset + '_'
                    + label] = datagen.flow_from_directory(
                        path_dataset_src[scenario_names[0] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label]
                    )
del dataset, label

##### Show image

In [ ]:
augment_image.visualize_img(datasets=dataset_names,
                                labels=labels_name,
                                datagen=s1_datagen,
                                aug=s1_aug,
                                col_mode=s1_col_mode,
                                scenario=scenario_names[0],
                                plot_dest=path_img)

#### Scenario 2
**Conditon:**
- basic augmentation
- rgb color
- no clahe

##### Prepare the needed variable

In [ ]:
s2_datagen = {}
s2_col_mode = 'rgb'
s2_aug = aug_type.copy()

##### Import the image into data generator

In [ ]:
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s2_datagen[dataset + '_'
                    + label] = datagen.flow_from_directory(
                        path_dataset_src[scenario_names[1] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label]
                    )
del dataset, label

##### Store image

In [ ]:
for augmentation_type in s2_aug:
    augment_image.visualize_img(datasets=dataset_names,
                                    labels=labels_name,
                                    datagen=s2_datagen,
                                    aug=augmentation_type,
                                    col_mode=s2_col_mode,
                                    scenario=scenario_names[1],
                                    plot_dest=path_img)
del augmentation_type

#### Scenario 3
**Condition:**
- basic augmentation
- rgb color
- clahe
##### Prepare the needed variable

In [ ]:
s3_datagen = {}
s3_datagen_clahe = {}
s3_col_mode = 'rgb'
s3_aug = aug_type.copy()

##### Import the image into data generator

In [ ]:
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s3_datagen[dataset + '_'
                    + label] = datagen.flow_from_directory(
                        path_dataset_src[scenario_names[2] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label],
                        shuffle=False
                    )
del dataset, label

In [ ]:
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s3_datagen_clahe[dataset + '_'
                    + label] = datagen_clahe.flow_from_directory(
                        path_dataset_src[scenario_names[2] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label],
                        shuffle=False
                    )
del dataset, label

##### Store image

In [ ]:
for augmentation_type in s3_aug:
    for dataset in dataset_names:
        # create the image size
        plt.figure(figsize=(9, 6),
                    dpi=300,
                    frameon=False,
                    layout='constrained')

        for img_place, label in enumerate(labels_name):
            # get 1 batch from the batched datagen
            images_def = next(s3_datagen[f'{dataset}_{label}'])
            images_clahe = next(s3_datagen_clahe[f'{dataset}_{label}'])

            if img_place == 0:
                # original image
                plt.subplot(2,3, img_place+1)
                plt.title(label=f'{label}',
                            fontdict={'fontsize': 14})
                plt.imshow(images_def[0][0])

                # image clahe implemented
                plt.subplot(2,3, img_place+2)
                plt.title(label=f'{label} clahe',
                            fontdict={'fontsize': 14})
                plt.imshow(images_clahe[0][0])

                # image clahe implemented and augmented
                plt.subplot(2,3, img_place+3)
                plt.imshow(augment_image.augment_img(image=images_clahe[0][0],
                                                    aug_type=augmentation_type))
                plt.title(label=f'{label} {augmentation_type}',
                            fontdict={'fontsize': 14})
            elif img_place == 1:
                # original image
                plt.subplot(2,3, img_place+3)
                plt.title(label=f'{label}',
                            fontdict={'fontsize': 14})
                plt.imshow(images_def[0][0])

                # image clahe implemented
                plt.subplot(2,3, img_place+4)
                plt.title(label=f'{label} clahe',
                            fontdict={'fontsize': 14})
                plt.imshow(images_clahe[0][0])

                # image clahe implemented and augmented
                plt.subplot(2,3, img_place+5)
                plt.imshow(augment_image.augment_img(image=images_clahe[0][0],
                                                    aug_type=augmentation_type))
                plt.title(label=f'{label} {augmentation_type}',
                            fontdict={'fontsize': 14})

        # saving the image
        plt.savefig(fname=os.path.join(path_img,
                                        f'{scenario_names[2]} {dataset} {augmentation_type}.png'),
                                        dpi=300,
                                        format='png',
                                        metadata={
                                            'Title': f'{dataset.title()} {augmentation_type.title()} Augmentation{scenario_names[2].replace("_", " ").title()}',
                                            'Author': 'Bugi Sulistiyo',
                                            'Description': f'Visualization of {dataset.title()} {augmentation_type.title()} Augmentation\n{scenario_names[2].replace("_", " ").title()}'
                                        })
del augmentation_type, dataset, label,
del images_def, images_clahe, img_place

#### Scenario 4
**Condition:**
- basic augmentation
- grayscale
- no clahe

##### Prepare the needed variable

In [ ]:
s4_datagen = {}
s4_col_mode = 'grayscale'
s4_aug = aug_type.copy()

##### Import the image into data generator

In [ ]:
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s4_datagen[dataset + '_'
                    + label] = datagen.flow_from_directory(
                        path_dataset_src[scenario_names[3] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label]
                    )
del dataset, label

##### Store image

In [ ]:
for augmentation_type in s4_aug:
    augment_image.visualize_img(datasets=dataset_names,
                                    labels=labels_name,
                                    datagen=s4_datagen,
                                    aug=augmentation_type,
                                    col_mode=s4_col_mode,
                                    scenario=scenario_names[3],
                                    plot_dest=path_img)
del augmentation_type

#### Scenario 5
**Condition:**
- basic augmentation
- grayscale
- clahe

##### Prepare the needed variable

In [ ]:
s5_datagen = {}
s5_datagen_clahe = {}
s5_col_mode = 'grayscale'
s5_aug = aug_type.copy()

##### Import the image into data generator

In [ ]:
# creating the rgb image data generator
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s5_datagen[dataset + '_'
                    + label] = datagen.flow_from_directory(
                        path_dataset_src[scenario_names[4] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label],
                        shuffle=False
                    )
del dataset, label

In [ ]:
# creating the gray image data generator
for dataset in dataset_names:
    for label in labels_name:
        print(f'Load {dataset} {label}...')
        s5_datagen_clahe[dataset + '_'
                    + label] = datagen_clahe.flow_from_directory(
                        path_dataset_src[scenario_names[4] + '_'
                                        + dataset + '_'
                                        + fold],
                        target_size=image_size[dataset],
                        class_mode='binary',
                        classes=[label],
                        shuffle=False,
                        color_mode=s5_col_mode
                    )
del dataset, label

##### Show image

In [ ]:
for augmentation_type in s5_aug:
    for dataset in dataset_names:
        # define the image sizee
        plt.figure(figsize=(9, 6),
                    dpi=300,
                    frameon=False,
                    layout='constrained')

        for img_place, label in enumerate(labels_name):
            # get 1 batch from the batched datagen
            images_def = next(s5_datagen[f'{dataset}_{label}'])
            images_clahe = next(s5_datagen_clahe[f'{dataset}_{label}'])

            if img_place == 0:
                # original image
                plt.subplot(2,3, img_place+1)
                plt.title(label=f'{label}',
                            fontdict={'fontsize': 14})
                plt.imshow(images_def[0][0])

                # image with clahe implemented
                plt.subplot(2,3, img_place+2)
                plt.title(label=f'{label} clahe',
                            fontdict={'fontsize': 14})
                plt.imshow(images_clahe[0][0], cmap='gray')

                # image with clahe implemented and augmented
                plt.subplot(2,3, img_place+3)
                plt.imshow(augment_image.augment_img(image=images_clahe[0][0],
                                                    aug_type=augmentation_type),
                            cmap='gray')
                plt.title(label=f'{label} {augmentation_type}',
                            fontdict={'fontsize': 14})
            elif img_place == 1:
                # original image
                plt.subplot(2,3, img_place+3)
                plt.title(label=f'{label}',
                            fontdict={'fontsize': 14})
                plt.imshow(images_def[0][0])

                # image with clahe implemented
                plt.subplot(2,3, img_place+4)
                plt.title(label=f'{label} clahe',
                            fontdict={'fontsize': 14})
                plt.imshow(images_clahe[0][0], cmap='gray')

                # image wit clahe implemented and augmented
                plt.subplot(2,3, img_place+5)
                plt.imshow(augment_image.augment_img(image=images_clahe[0][0],
                                                    aug_type=augmentation_type),
                            cmap='gray')
                plt.title(label=f'{label} {augmentation_type}',
                            fontdict={'fontsize': 14})

        # save the image
        plt.savefig(fname=os.path.join(path_img,
                                        f'{scenario_names[4]} {dataset} {augmentation_type}.png'),
                                        dpi=300,
                                        format='png',
                                        metadata={
                                            'Title': f'{dataset.title()} {augmentation_type.title()} Augmentation{scenario_names[4].replace("_", " ").title()}',
                                            'Author': 'Bugi Sulistiyo',
                                            'Description': f'Visualization of {dataset.title()} {augmentation_type.title()} Augmentation\n{scenario_names[4].replace("_", " ").title()}'
                                        })
del augmentation_type, dataset, label
del images_def, images_clahe, img_place